# EDA for Walmart Weekly Sale

In [260]:
import pandas as pd
import numpy as np
from numpy import *

import gc
import time
import warnings
import os

In [261]:
data_dir = 'data'
train_file_name = 'train.csv'
features_file_name = 'features.csv'
store_file_name = 'stores.csv'


In [262]:
train_data = pd.read_csv(os.path.join(data_dir,train_file_name))
train_data.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [263]:
feature_data = pd.read_csv(os.path.join(data_dir,features_file_name))
feature_data.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [264]:
store_data = pd.read_csv(os.path.join(data_dir,store_file_name))
store_data.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [265]:
store_data.Type.unique()

array(['A', 'B', 'C'], dtype=object)

# EDA for Store

In [266]:
train_df = train_data.copy()

In [267]:
train_df.Store.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45])

In [268]:
train_df.Dept.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 40, 41, 42, 44, 45, 46, 47, 48, 49, 51, 52, 54, 55, 56,
       58, 59, 60, 67, 71, 72, 74, 77, 78, 79, 80, 81, 82, 83, 85, 87, 90,
       91, 92, 93, 94, 95, 96, 97, 98, 99, 39, 50, 43, 65])

In [271]:
print("the number of weeks: ", train_df['Date'].nunique())
train_df['Date'].unique()

the number of weeks:  143


array(['2010-02-05', '2010-02-12', '2010-02-19', '2010-02-26',
       '2010-03-05', '2010-03-12', '2010-03-19', '2010-03-26',
       '2010-04-02', '2010-04-09', '2010-04-16', '2010-04-23',
       '2010-04-30', '2010-05-07', '2010-05-14', '2010-05-21',
       '2010-05-28', '2010-06-04', '2010-06-11', '2010-06-18',
       '2010-06-25', '2010-07-02', '2010-07-09', '2010-07-16',
       '2010-07-23', '2010-07-30', '2010-08-06', '2010-08-13',
       '2010-08-20', '2010-08-27', '2010-09-03', '2010-09-10',
       '2010-09-17', '2010-09-24', '2010-10-01', '2010-10-08',
       '2010-10-15', '2010-10-22', '2010-10-29', '2010-11-05',
       '2010-11-12', '2010-11-19', '2010-11-26', '2010-12-03',
       '2010-12-10', '2010-12-17', '2010-12-24', '2010-12-31',
       '2011-01-07', '2011-01-14', '2011-01-21', '2011-01-28',
       '2011-02-04', '2011-02-11', '2011-02-18', '2011-02-25',
       '2011-03-04', '2011-03-11', '2011-03-18', '2011-03-25',
       '2011-04-01', '2011-04-08', '2011-04-15', '2011-

### Notice: The date below starts from Friday 
**However Amazon Forecast's Week starts from Monday as far as I know. we might handle this**

In [214]:
train_df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


### Shift three days in the future because of a difference of starting weekday against Amazon Forecast

In [215]:

train_df.Date = pd.to_datetime(train_df['Date']) + pd.DateOffset(days=3)


### Rename 'Store' with item_id for an intuitive purpose on Amazon Forecast

In [216]:

train_df = train_df.rename(columns={'Date':'date', 'Store':'item_id'})
train_df.head()

,item_id,Dept,date,Weekly_Sales,IsHoliday
0,1,1,2010-02-08,24924.50,False
1,1,1,2010-02-15,46039.49,True
2,1,1,2010-02-22,41595.55,False
3,1,1,2010-03-01,19403.54,False
4,1,1,2010-03-08,21827.90,False


In [217]:
train_df.isnull().sum()

item_id         0
Dept            0
date            0
Weekly_Sales    0
IsHoliday       0
dtype: int64

In [218]:
train_df.groupby(['item_id','date','Dept'])['Weekly_Sales'].sum().head()

item_id  date        Dept
1        2010-02-08  1       24924.50
                     2       50605.27
                     3       13740.12
                     4       39954.04
                     5       32229.38
Name: Weekly_Sales, dtype: float64

#### To simplify this problem, aggregate Weekly_Sales by item_id and date,making an effect of removing 'Dept'

In [219]:
train_df.head()

,item_id,Dept,date,Weekly_Sales,IsHoliday
0,1,1,2010-02-08,24924.50,False
1,1,1,2010-02-15,46039.49,True
2,1,1,2010-02-22,41595.55,False
3,1,1,2010-03-01,19403.54,False
4,1,1,2010-03-08,21827.90,False


In [220]:
temp_df = train_df.groupby(['item_id','date'])['Weekly_Sales'].sum()
stores_sales = pd.DataFrame(temp_df)
stores_sales.head()

Weekly_Sales
item_id date                    
1       2010-02-08    1643690.90
        2010-02-15    1641957.44
        2010-02-22    1611968.17
        2010-03-01    1409727.59
        2010-03-08    1554806.68

In [221]:
# Check if a condition is okay with "item_id" == 1, "date" == 2010-02-08
def spot_check(train_df):
    agg_check = train_df.groupby(['item_id','date','Dept'])['Weekly_Sales'].sum()
    agg_check = agg_check.reset_index()
    filter = (agg_check['item_id'] == 1) & (agg_check['date'] == '2010-02-08')
    total_sum = agg_check[filter]['Weekly_Sales'].sum() 
    print("total sum across department is :", total_sum)
    assert ( total_sum == 1643690.90)
    
spot_check(train_df)

total sum across department is : 1643690.9


In [222]:
stores_sales.reset_index(inplace=True)
stores_sales = stores_sales.set_index('date')
stores_sales.head()
print(stores_sales.index.max())
print(stores_sales.index.min())


2012-10-29 00:00:00
2010-02-08 00:00:00


#### Change type to suit the target time series data set

In [223]:

stores_sales.item_id = stores_sales.item_id.astype(str)
stores_sales.index = pd.to_datetime(stores_sales.index, format = '%Y-%m-%d' )
#stores_sales.index = pd.to_datetime(stores_sales.index.strftime("yyyy-MM-dd hh:mm:ss"))
cols = ['Weekly_Sales', 'item_id']
stores_sales = stores_sales[cols]

In [224]:
stores_sales.head()

,Weekly_Sales,item_id
date,,
2010-02-08,1643690.90,1
2010-02-15,1641957.44,1
2010-02-22,1611968.17,1
2010-03-01,1409727.59,1
2010-03-08,1554806.68,1


## Period of Train and Validation

In [225]:
end_train_date = '2012-07-30' # Non-inclusive
end_val_date = '2012-10-29'

In [226]:

target_stores_sales = stores_sales[stores_sales.index < end_train_date]
validation_stores_sales = stores_sales[stores_sales.index >= end_train_date]
validation_stores_sales = validation_stores_sales[validation_stores_sales.index < end_val_date]


In [227]:
target_stores_sales.tail()

,Weekly_Sales,item_id
date,,
2012-06-25,822569.16,45
2012-07-02,773367.71,45
2012-07-09,843361.10,45
2012-07-16,749817.08,45
2012-07-23,737613.65,45


In [228]:
# With the data in a great state, save it off as a CSV
target_time_series_filename = "target_time_series.csv"
target_time_series_path = data_dir + "/" + target_time_series_filename
target_stores_sales.to_csv(target_time_series_path, header=False)

In [229]:
# With the data in a great state, save it off as a CSV
validation_time_series_filename = "validation_time_series.csv"
validation_time_series_path = data_dir + "/" + validation_time_series_filename
validation_stores_sales.to_csv(validation_time_series_path, header=False)

In [230]:
target_stores_sales.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5805 entries, 2010-02-08 to 2012-07-23
Data columns (total 2 columns):
Weekly_Sales    5805 non-null float64
item_id         5805 non-null object
dtypes: float64(1), object(1)
memory usage: 136.1+ KB


In [231]:
validation_stores_sales.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 585 entries, 2012-07-30 to 2012-10-22
Data columns (total 2 columns):
Weekly_Sales    585 non-null float64
item_id         585 non-null object
dtypes: float64(1), object(1)
memory usage: 13.7+ KB


In [232]:
target_stores_sales.head()

,Weekly_Sales,item_id
date,,
2010-02-08,1643690.90,1
2010-02-15,1641957.44,1
2010-02-22,1611968.17,1
2010-03-01,1409727.59,1
2010-03-08,1554806.68,1


In [233]:
target_stores_sales.groupby('item_id').count()

,Weekly_Sales
item_id,
1,129
10,129
11,129
12,129
13,129
14,129
15,129
16,129
17,129


# EDA for Features

In [234]:
feature_df = feature_data.copy()
feature_df.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [235]:
# Shift three days in the future because of a difference of starting weekday against Amazon Forecast
feature_df.Date = pd.to_datetime(feature_df['Date']) + pd.DateOffset(days=3)


In [236]:
feature_df.isnull().sum()

Store              0
Date               0
Temperature        0
Fuel_Price         0
MarkDown1       4158
MarkDown2       5269
MarkDown3       4577
MarkDown4       4726
MarkDown5       4140
CPI              585
Unemployment     585
IsHoliday          0
dtype: int64

In [237]:
# Drop MarkDown columns 
feature_df = feature_df.drop(['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5'], axis=1)

In [238]:
feature_df.head()

,Store,Date,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday
0,1,2010-02-08,42.31,2.572,211.096358,8.106,False
1,1,2010-02-15,38.51,2.548,211.242170,8.106,True
2,1,2010-02-22,39.93,2.514,211.289143,8.106,False
3,1,2010-03-01,46.63,2.561,211.319643,8.106,False
4,1,2010-03-08,46.50,2.625,211.350143,8.106,False


In [239]:
feature_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 7 columns):
Store           8190 non-null int64
Date            8190 non-null datetime64[ns]
Temperature     8190 non-null float64
Fuel_Price      8190 non-null float64
CPI             7605 non-null float64
Unemployment    7605 non-null float64
IsHoliday       8190 non-null bool
dtypes: bool(1), datetime64[ns](1), float64(4), int64(1)
memory usage: 392.0 KB


In [240]:
# Drop CPI, Unemployment because of missing values
# Drop isHoliday because isHoliday exists in the built-in data of AWS Forecast
feature_df = feature_df.drop(['CPI','Unemployment','IsHoliday'], axis=1)

In [241]:
feature_df.head()

,Store,Date,Temperature,Fuel_Price
0,1,2010-02-08,42.31,2.572
1,1,2010-02-15,38.51,2.548
2,1,2010-02-22,39.93,2.514
3,1,2010-03-01,46.63,2.561
4,1,2010-03-08,46.50,2.625


#### Change a format of the feature

In [242]:
feature_df = feature_df.rename(columns={'Store':'item_id','Date':'date'})
feature_df.reset_index(inplace=True)
feature_df = feature_df.drop('index', axis=1)
feature_df = feature_df.set_index('date')
feature_df.item_id = feature_df.item_id.astype(str)
feature_df.index = pd.to_datetime(feature_df.index, format = '%Y-%m-%d')
cols = ['Temperature','Fuel_Price','item_id']
feature_df = feature_df[cols]
feature_df.head()

,Temperature,Fuel_Price,item_id
date,,,
2010-02-08,42.31,2.572,1
2010-02-15,38.51,2.548,1
2010-02-22,39.93,2.514,1
2010-03-01,46.63,2.561,1
2010-03-08,46.50,2.625,1


In [243]:
feature_df = feature_df[feature_df.index < end_val_date]
feature_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6390 entries, 2010-02-08 to 2012-10-22
Data columns (total 3 columns):
Temperature    6390 non-null float64
Fuel_Price     6390 non-null float64
item_id        6390 non-null object
dtypes: float64(2), object(1)
memory usage: 199.7+ KB


In [244]:
feature_df.isnull().sum()

Temperature    0
Fuel_Price     0
item_id        0
dtype: int64

In [245]:
feature_df.tail()

,Temperature,Fuel_Price,item_id
date,,,
2012-09-24,65.32,4.038,45
2012-10-01,64.88,3.997,45
2012-10-08,64.89,3.985,45
2012-10-15,54.47,4.000,45
2012-10-22,56.47,3.969,45


In [246]:
print(feature_df.index.min())
print(feature_df.index.max())


2010-02-08 00:00:00
2012-10-22 00:00:00


In [247]:
pd.merge(target_stores_sales, feature_df, 
         left_index=True, right_index=True,
        how='outer')

,Weekly_Sales,item_id_x,Temperature,Fuel_Price,item_id_y
date,,,,,
2010-02-08,1643690.9,1,42.31,2.572,1
2010-02-08,1643690.9,1,40.19,2.572,2
2010-02-08,1643690.9,1,45.71,2.572,3
2010-02-08,1643690.9,1,43.76,2.598,4
2010-02-08,1643690.9,1,39.70,2.572,5
2010-02-08,1643690.9,1,40.43,2.572,6
2010-02-08,1643690.9,1,10.53,2.580,7
2010-02-08,1643690.9,1,34.14,2.572,8
2010-02-08,1643690.9,1,38.01,2.572,9


# EDA for Store

In [248]:
store_df = store_data.copy()

In [249]:
store_df = store_df.drop('Size', axis=1)
store_df = store_df.rename(columns={'Store':'item_id', 'Type':'StoreType'})

In [250]:
store_df = store_df.set_index('item_id')

In [251]:
store_df

,StoreType
item_id,
1,A
2,A
3,B
4,A
5,B
6,A
7,B
8,A
9,B
